In [1]:
!pip install -U "pip<25.3"
!pip install -U "numpy==1.26.4" \
               "pandas==2.2.2" \
               "matplotlib==3.8.4" \
               "pillow==10.3.0" \
               "scikit-learn==1.4.2" \
               "opencv-python==4.10.0.84"


Defaulting to user installation because normal site-packages is not writeable
  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\ProgramData\anaconda3\python.exe -m pip install -U pip<25.3


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os, json, glob
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# 그래프 한글 폰트 이슈가 있으면 주석 해제하고 시스템에 맞는 폰트를 지정하세요.
# import matplotlib
# matplotlib.rc('font', family='Malgun Gothic')  # 윈도우 '맑은 고딕'
# plt.rcParams['axes.unicode_minus'] = False


In [3]:
# Windows 한글 폰트 설정 (맑은 고딕)
from matplotlib import rcParams, font_manager
try:
    font_manager.fontManager.addfont(r"C:\Windows\Fonts\malgun.ttf")
    rcParams["font.family"] = "Malgun Gothic"
    rcParams["axes.unicode_minus"] = False
    print("Matplotlib font set to Malgun Gothic.")
except Exception as e:
    print("폰트 설정 실패:", e)


Matplotlib font set to Malgun Gothic.


In [4]:
\
# ★★★ 여기만 확인/수정하세요 ★★★
# 윈도우의 실제 데이터 폴더
BASE_DIR = Path(r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent")

# 서브 폴더들 (변경하지 않아도 됨)
LABELS_DIR = BASE_DIR / "labels"                # JSON들이 들어있는 폴더
EMOTION_DIR = BASE_DIR                           # 감정별 이미지 폴더들이 바로 아래에 있음 (중립/슬픔/상처/불안/분노/당황/기쁨)
PROCESSED_DIR = BASE_DIR / "processed"           # 전처리 결과 저장 폴더(자동 생성)
(PROCESSED_DIR / "from_json").mkdir(parents=True, exist_ok=True)
(PROCESSED_DIR / "from_folders").mkdir(parents=True, exist_ok=True)

BASE_DIR, LABELS_DIR, EMOTION_DIR, PROCESSED_DIR


(WindowsPath('C:/Users/USER/Downloads/korean_emotion_complex_vision_1_percent'),
 WindowsPath('C:/Users/USER/Downloads/korean_emotion_complex_vision_1_percent/labels'),
 WindowsPath('C:/Users/USER/Downloads/korean_emotion_complex_vision_1_percent'),
 WindowsPath('C:/Users/USER/Downloads/korean_emotion_complex_vision_1_percent/processed'))

### 1) 감정 폴더 스캔

In [5]:
emotions = []
for p in EMOTION_DIR.iterdir():
    if p.is_dir() and p.name not in ['labels', 'processed']:
        emotions.append(p.name)
emotions


['samples', '기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']

### 2) 224×224 전처리 저장

In [ ]:
out_root = PROCESSED_DIR / "from_folders"
out_root.mkdir(exist_ok=True)
records = []

def resize_center_square(im, target=224):
    w,h = im.size
    scale = target / min(w,h)
    nw, nh = int(w*scale), int(h*scale)
    im2 = im.resize((nw,nh), Image.BICUBIC)
    left = max(0, (nw - target)//2)
    top = max(0, (nh - target)//2)
    return im2.crop((left, top, left+target, top+target))

for em in emotions:
    src_dir = EMOTION_DIR / em
    dst_dir = out_root / em
    dst_dir.mkdir(parents=True, exist_ok=True)
    for ext in ('*.jpg','*.jpeg','*.png','*.bmp','*.webp'):
        for p in src_dir.glob(ext):
            try:
                with Image.open(p) as im:
                    im = im.convert('RGB')
                    proc = resize_center_square(im, target=224)
                out_path = dst_dir / (p.stem + ".jpg")
                proc.save(out_path, quality=90)
                records.append({'file': str(out_path), 'label': em})
            except Exception as e:
                print("처리 실패:", p, e)

df_fold = pd.DataFrame(records)
df_fold.head(), len(df_fold)


### 3) CSV 저장

In [ ]:
out_csv = BASE_DIR / "processed_folders_labels.csv"
df_fold.to_csv(out_csv, index=False)
out_csv
